In [1]:
MAX_ROUNDS = 400
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50  

In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from numba import jit
import time
import gc
import warnings
warnings.filterwarnings("ignore")

불순도낮다 = 분류가 잘 되었다.

In [3]:
# https://www.kaggle.com/cpmpml/extremely-fast-gini-computation

def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

In [4]:
# https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = -eval_gini(labels, preds)
    return [('gini', gini_score)]


def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))
  
def target_encode(trn_series=None,  
                  val_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing은 Daniele Micci-Barreca의 논문에서와 같이 계산됩니다.
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : pd.Series 형태의 학습할 범주형 피처
    tst_series : pd.Series 형태의 테스트할 범주형 피처
    target : pd.Series 형태의 타겟 데이터
    min_samples_leaf (int) : 범주의 평균을 고려할 최소 샘플
    smoothing (int) : 범주 평균과 이전의 균형을 맞추기 위한 스무딩 효과 
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    
    # 타겟 평균을 계산
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    
    # 스무딩 계산
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    
    # 모든 타겟 데이터에 평균 적용하기
    prior = target.mean()
    
    # 카운트가 클수록 full_avg가 적어집니다.
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    
    # 학습, 테스트 데이터에 평균 적용
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge는 인덱스를 유지하지 않으므로 복원합니다.
    ft_trn_series.index = trn_series.index
    ft_val_series = pd.merge(
        val_series.to_frame(val_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=val_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge는 인덱스를 유지하지 않으므로 복원합니다.
    ft_val_series.index = val_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge는 인덱스를 유지하지 않으므로 복원합니다.
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_val_series, noise_level), add_noise(ft_tst_series, noise_level)

In [5]:
# 데이터 읽기
train_df = pd.read_csv("data/train.csv", na_values="-1")
test_df = pd.read_csv("data/test.csv", na_values="-1")

In [6]:
# olivier 커널 참고
train_features = [
  "ps_car_13",       #            : 1571.65 / shadow  609.23
  "ps_reg_03",       #            : 1408.42 / shadow  511.15
  "ps_ind_05_cat",   #        : 1387.87 / shadow   84.72
  "ps_ind_03",       #            : 1219.47 / shadow  230.55
  "ps_ind_15",       #            :  922.18 / shadow  242.00
  "ps_reg_02",       #            :  920.65 / shadow  267.50
  "ps_car_14",       #            :  798.48 / shadow  549.58
  "ps_car_12",       #            :  731.93 / shadow  293.62
  "ps_car_01_cat",   #        :  698.07 / shadow  178.72
  "ps_car_07_cat",   #        :  694.53 / shadow   36.35
  "ps_ind_17_bin",   #        :  620.77 / shadow   23.15
  "ps_car_03_cat",   #        :  611.73 / shadow   50.67
  "ps_reg_01",       #            :  598.60 / shadow  178.57
  "ps_car_15",       #            :  593.35 / shadow  226.43
  "ps_ind_01",       #            :  547.32 / shadow  154.58
  "ps_ind_16_bin",   #        :  475.37 / shadow   34.17
  "ps_ind_07_bin",   #        :  435.28 / shadow   28.92
  "ps_car_06_cat",   #        :  398.02 / shadow  212.43
  "ps_car_04_cat",   #        :  376.87 / shadow   76.98
  "ps_ind_06_bin",   #        :  370.97 / shadow   36.13
  "ps_car_09_cat",   #        :  214.12 / shadow   81.38
  "ps_car_02_cat",   #        :  203.03 / shadow   26.67
  "ps_ind_02_cat",   #        :  189.47 / shadow   65.68
  "ps_car_11",       #            :  173.28 / shadow   76.45
  "ps_car_05_cat",   #        :  172.75 / shadow   62.92
  "ps_calc_09",      #           :  169.13 / shadow  129.72
  "ps_calc_05",      #           :  148.83 / shadow  120.68
  "ps_ind_08_bin",   #        :  140.73 / shadow   27.63
  "ps_car_08_cat",   #        :  120.87 / shadow   28.82
  "ps_ind_09_bin",   #         :  113.92 / shadow   27.05
  "ps_ind_04_cat",   #        :  107.27 / shadow   37.43
  "ps_ind_18_bin",   #        :   77.42 / shadow   25.97
  "ps_ind_12_bin",   #        :   39.67 / shadow   15.52
  "ps_ind_14",       #            :   37.37 / shadow   16.65
]
# 조합 추가
combs = [
    ('ps_reg_01', 'ps_car_02_cat'),  
    ('ps_reg_01', 'ps_car_04_cat'),
]

In [7]:
for n_c, (f1, f2) in enumerate(combs):
    print(n_c, (f1, f2))

0 ('ps_reg_01', 'ps_car_02_cat')
1 ('ps_reg_01', 'ps_car_04_cat')


In [8]:
# train_df[ps_reg_01_plus_ps_car_04_cat] = train_df['ps_reg_01].apply(lambda x: str(x)) + "_" + train_df[f2].apply(lambda x: str(x))


In [9]:
# 데이터 처리
id_train = train_df['id'].values
id_test = test_df['id'].values

y = train_df['target']

start = time.time()   # 현재 시간
for n_c, (f1, f2) in enumerate(combs):
    name1 = f1 + "_plus_" + f2
    print('current feature %60s %4d in %5.1f'
          % (name1, n_c + 1, (time.time() - start) / 60), end='')
    print('\r' * 75, end='')
    train_df[name1] = train_df[f1].apply(lambda x: str(x)) + "_" + train_df[f2].apply(lambda x: str(x))
    test_df[name1] = test_df[f1].apply(lambda x: str(x)) + "_" + test_df[f2].apply(lambda x: str(x))
    print(train_df[name1])
    
    # Label Encode
    lbl = LabelEncoder()
    lbl.fit(list(train_df[name1].values) + list(test_df[name1].values))
    train_df[name1] = lbl.transform(list(train_df[name1].values))
    test_df[name1] = lbl.transform(list(test_df[name1].values))

    train_features.append(name1)
    
X = train_df[train_features]
test_df = test_df[train_features]

f_cats = [f for f in X.columns if "_cat" in f]

0         0.7_1.0                               ps_reg_01_plus_ps_car_02_cat    1 in   0.0
1         0.8_1.0
2         0.0_1.0
3         0.9_1.0
4         0.7_1.0
           ...   
595207    0.5_1.0
595208    0.9_1.0
595209    0.9_1.0
595210    0.9_1.0
595211    0.1_0.0
Name: ps_reg_01_plus_ps_car_02_cat, Length: 595212, dtype: object
current feature                                 ps_reg_01_plus_ps_car_04_cat    2 in   0.10         0.7_0
1         0.8_0
2         0.0_0
3         0.9_0
4         0.7_0
          ...  
595207    0.5_0
595208    0.9_0
595209    0.9_0
595210    0.9_0
595211    0.1_0
Name: ps_reg_01_plus_ps_car_04_cat, Length: 595212, dtype: object


In [10]:
train_features

['ps_car_13',
 'ps_reg_03',
 'ps_ind_05_cat',
 'ps_ind_03',
 'ps_ind_15',
 'ps_reg_02',
 'ps_car_14',
 'ps_car_12',
 'ps_car_01_cat',
 'ps_car_07_cat',
 'ps_ind_17_bin',
 'ps_car_03_cat',
 'ps_reg_01',
 'ps_car_15',
 'ps_ind_01',
 'ps_ind_16_bin',
 'ps_ind_07_bin',
 'ps_car_06_cat',
 'ps_car_04_cat',
 'ps_ind_06_bin',
 'ps_car_09_cat',
 'ps_car_02_cat',
 'ps_ind_02_cat',
 'ps_car_11',
 'ps_car_05_cat',
 'ps_calc_09',
 'ps_calc_05',
 'ps_ind_08_bin',
 'ps_car_08_cat',
 'ps_ind_09_bin',
 'ps_ind_04_cat',
 'ps_ind_18_bin',
 'ps_ind_12_bin',
 'ps_ind_14',
 'ps_reg_01_plus_ps_car_02_cat',
 'ps_reg_01_plus_ps_car_04_cat']

In [11]:
y_valid_pred = 0*y
y_test_pred = 0

In [12]:
# 폴드 설정
K = 5
kf = KFold(n_splits = K, random_state = 1, shuffle = True)
np.random.seed(0)

XGBClassifier 파라미터
- **min_child_weight**(반드시 튜닝)
  - 값이 높아지면 under-fitting 되는 경우가 있다.
  - CV를 통해 튜닝되어야 한다.
- **max_depth**(반드시 튜닝)
  - 트리의 최대 깊이를 정의
  - 루트에서 가장 긴 노드의 거리
  - Typical Value는 3-10
- **gamma**(반드시 튜닝)
  - 노드가 split 되기 위한 loss function의 값이 감소하는 최소값을 정의
  - gamma 값이 높아질 수록 알고리즘은 보수적으로 변한다.
  - loss function의 정의에 따라 적정값이 달라지기때문에 반드시 튜닝해야 한다.
- **colsample_bytree**
  - 트리를 생성할때 훈련 데이터에서 변수를 샘플링해주는 비율. 보통0.6~0.9
- **objective** = 'reg:linear','binary:logistic','multi:softmax','multi:softprob'   - 4가지 존재
  -  회귀 경우 'reg'
  - binary분류의 경우 'binary',
  - 다중분류경우 'multi'- 
    - 분류된 class를 return하는 경우 'softmax'
    - 각 class에 속할 확률을 return하는 경우 'softprob'
    

In [13]:
## 분류기 설정
model = XGBClassifier(    
                        n_estimators=MAX_ROUNDS,
                        max_depth=4,
                        objective="binary:logistic",
                        learning_rate=LEARNING_RATE, 
                        subsample=.8,
                        min_child_weight=6,
                        colsample_bytree=.8,
                        scale_pos_weight=1.6,
                        gamma=10,
                        reg_alpha=8,
                        reg_lambda=1.3,
                     )

In [ ]:
# CV 실행
for i, (train_index, test_index) in enumerate(kf.split(train_df)):
    
    # 폴드에 대한 데이터 생성
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index,:].copy(), X.iloc[test_index,:].copy()
    X_test = test_df.copy()
    print( "\nFold ", i)
    
    # 데이터 인코드
    for f in f_cats:
        X_train[f + "_avg"], X_valid[f + "_avg"], X_test[f + "_avg"] = target_encode(
                                                        trn_series=X_train[f],
                                                        val_series=X_valid[f],
                                                        tst_series=X_test[f],
                                                        target=y_train,
                                                        min_samples_leaf=200,
                                                        smoothing=10,
                                                        noise_level=0
                                                        )
    # 폴드에 대한 모델 실행
    if OPTIMIZE_ROUNDS:
        eval_set=[(X_valid,y_valid)]
        fit_model = model.fit( X_train, y_train, 
                               eval_set=eval_set,
                               eval_metric=gini_xgb,
                               early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                               verbose=False
                             )
        print( "  Best N trees = ", model.best_ntree_limit )
        print( "  Best gini = ", model.best_score )
    else:
        fit_model = model.fit( X_train, y_train )
        
    # 폴드에 대한 검증 예측 생성
    pred = fit_model.predict_proba(X_valid)[:,1]
    print( "  Gini = ", eval_gini(y_valid, pred) )
    y_valid_pred.iloc[test_index] = pred
    
    # 테스트 세트의 예측을 누적
    y_test_pred += fit_model.predict_proba(X_test)[:,1]
    
    del X_test, X_train, X_valid, y_train
    
y_test_pred /= K  # 테스트 세트 예측의 평균

print( "\nGini for full training set:" )
eval_gini(y, y_valid_pred)


Fold  0
  Gini =  0.28468979165163677

Fold  1
  Gini =  0.2825062745155765

Fold  2
  Gini =  0.2741647405553511

Fold  3


In [ ]:
# 스태킹 / 앙상블을위한 검증 예측 저장
val = pd.DataFrame()
val['id'] = id_train
val['target'] = y_valid_pred.values
val.to_csv(DATA_PATH + 'xgb_valid.csv', float_format='%.6f', index=False)

In [ ]:
## 제출 파일 생성
sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = y_test_pred
sub.to_csv(DATA_PATH + 'xgb_submit.csv', float_format='%.6f', index=False)